In [1]:
%pip install streamlit
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install plotly
%pip install statsmodels

#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
! npm install localtunnel


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 in

In [39]:
%%writefile EVIDENCIA_2.py

## streamlit run EVIDENCIA_2.py

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import streamlit as st

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, roc_curve
)
from scipy.optimize import curve_fit

# =========================
# Utilidades base
# =========================
def safe_read_csv(path):
    try:
        if os.path.exists(path):
            return pd.read_csv(path)
    except Exception:
        pass
    return None

def unify_schema(df, city_name=None, verbose=False):
    """
    Normaliza nombres, resuelve columnas duplicadas, agrega 'city',
    y convierte strings con '%' a numéricos. Evita AttributeError.
    """
    if df is None or len(df) == 0:
        return None

    # 1) Normaliza nombres a snake_case
    cols_raw = list(df.columns)
    norm_cols = [str(c).strip().lower().replace(" ", "_") for c in cols_raw]

    # 2) Unifica duplicados: col, col__1, col__2, ...
    seen = {}
    unique_cols = []
    dups = []
    for c in norm_cols:
        if c not in seen:
            seen[c] = 0
            unique_cols.append(c)
        else:
            seen[c] += 1
            new_c = f"{c}__{seen[c]}"
            unique_cols.append(new_c)
            dups.append((c, new_c))
    if verbose and dups:
        print("Columnas duplicadas renombradas:", dups)

    df = df.copy()
    df.columns = unique_cols

    # 3) Correcciones habituales
    ren = {
        "accomodates": "accommodates",
        "bedroms": "bedrooms",
        "bathroom": "bathrooms",
        "host_response_time(%)": "host_response_rate",
        "review_scores_value": "review_scores_rating"
    }
    for k, v in ren.items():
        if k in df.columns and v not in df.columns:
            df.rename(columns={k: v}, inplace=True)

    # 4) Asegura 'city'
    if "city" not in df.columns:
        df["city"] = city_name if city_name else "unknown"
    else:
        if city_name:
            df["city"] = df["city"].fillna(city_name).replace("", city_name)

    # 5) Convierte columnas con símbolo %
    for c in df.columns:
        s = df[c]
        if pd.api.types.is_object_dtype(s):
            if s.astype(str).str.contains("%").any():
                s2 = s.astype(str).str.replace("%", "", regex=False)
                s2 = pd.to_numeric(s2, errors="coerce")
                df[c] = s2

    return df

def numeric_cols(df):
    return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

def categorical_cols(df):
    return [c for c in df.columns if (df[c].dtype == "object" or str(df[c].dtype).startswith("category"))]

def r_from_y_ypred(y_true, y_pred):
    if len(y_true) < 2 or len(y_pred) < 2:
        return np.nan
    try:
        return float(np.corrcoef(y_true, y_pred)[0,1])
    except Exception:
        return np.nan

def fit_nonlinear_series(x, y, model_name):
    def f_quad(z, a, b, c): return a*z**2 + b*z + c
    def f_exp(z, a, b, c):  return a*np.exp(b*z) + c
    def f_inv(z, a, b):     return a + b/np.where(z==0, np.finfo(float).eps, z)

    if model_name == "Cuadrática":
        popt, _ = curve_fit(f_quad, x, y, maxfev=20000)
        yhat = f_quad(x, *popt)
        return yhat, popt
    elif model_name == "Exponencial":
        p0 = (1.0, 0.01, float(np.nanmedian(y)))
        popt, _ = curve_fit(f_exp, x, y, p0=p0, maxfev=20000)
        yhat = f_exp(x, *popt)
        return yhat, popt
    else:  # Inversa
        popt, _ = curve_fit(f_inv, x, y, maxfev=20000)
        yhat = f_inv(x, *popt)
        return yhat, popt

def coerce_numeric_cols(df, cols):
    """Devuelve una copia donde 'cols' se convierten a numérico con errors='coerce'."""
    d = df.copy()
    for c in cols:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors='coerce')
    return d

# =========================
# Configuración app
# =========================
st.set_page_config(
    page_title="Dashboard Mult-ciudad Airbnb: Proyecto Innovativo Grupo IPYNB",
    page_icon="🏠",
    layout="wide"
)

# --- Portada (Imagen + Título + Descripción) ---
# Ruta proporcionada por el usuario (cámbiala si tu archivo está en otra ubicación)
PORTADA_PATHS = [
    "WhatsApp Image 2025-10-22 at 11.38.03 PM.jpeg",
    "./assets/aura.jpg",
    "./assets/aura.png",
    "/mnt/data/WhatsApp Image 2025-10-22 at 11.38.03 PM.jpeg",
]
img_path = next((p for p in PORTADA_PATHS if os.path.exists(p)), None)
if img_path:
    col_l, col_c, col_r = st.columns([1,2,1])
    with col_c:
        st.image(img_path, use_column_width=True)
else:
    st.info("⚠️ Imagen de portada no encontrada. Verifica la ruta en PORTADA_PATHS.")

st.title("Dashboard Mult-ciudad Airbnb: Proyecto Innovativo Grupo IPYNB")
st.write("**Este dashboard permite establecer un panorama completo de Airbnb, sus Hosts y Alojamientos, potencializando al Programa AURA.**")

# =========================
# Sidebar
# =========================
with st.sidebar:
    st.header("⚙️ Configuración")
    color = st.color_picker("Color principal", "#0B6E4F")
    grid_alpha = st.slider("Transparencia de rejilla", 0.0, 1.0, 0.2, 0.05)
    st.markdown("---")
    st.subheader("📂 Carga de datos (opcional)")
    ups = st.file_uploader("Sube CSV (puedes múltiples)", type=["csv"], accept_multiple_files=True)

# =========================
# Carga de datasets base
# =========================
df_mex = unify_schema(safe_read_csv("Mexico_Limpio_modified.csv"), "Mexico")
df_mad = unify_schema(safe_read_csv("Madrid_AirBnb_010.csv"), "Madrid")
df_val = unify_schema(safe_read_csv("VALENCIA_LIMPIO.csv"), "Valencia")
df_rio = unify_schema(safe_read_csv("base_final.csv"), "Rio")   # <- etiqueta "Rio"
df_roma = unify_schema(safe_read_csv("df_limp_ROMA.csv"), "Roma")

dfs_raw = [d for d in [df_mex, df_mad, df_val, df_rio, df_roma] if d is not None]

if ups:
    for f in ups:
        try:
            tmp = pd.read_csv(f)
            dfs_raw.append(unify_schema(tmp, city_name=os.path.splitext(f.name)[0]))
        except Exception as e:
            st.warning(f"No se pudo leer {f.name}: {e}")

if not dfs_raw:
    st.error("No se encontraron datos. Coloca los CSV en la carpeta o súbelos.")
    st.stop()

# Intersección de columnas comunes para asegurar comparabilidad
common_cols = set(dfs_raw[0].columns)
for d in dfs_raw[1:]:
    common_cols = common_cols.intersection(set(d.columns))
common_cols = sorted(list(common_cols))

# Reindexa cada DF a columnas comunes
dfs = [d[common_cols].copy() for d in dfs_raw]

# Concat para análisis global
df_all = pd.concat(dfs, ignore_index=True)
cities_all = sorted(df_all["city"].dropna().astype(str).unique().tolist())

# Selección de ciudades a comparar
sel_cities = st.multiselect("🌍 Ciudades a comparar", options=cities_all, default=cities_all)
if not sel_cities:
    st.warning("Selecciona al menos una ciudad.")
    st.stop()
df = df_all[df_all["city"].isin(sel_cities)].reset_index(drop=True)

# Columnas por tipo (en universo común)
num_cols_all = [c for c in common_cols if pd.api.types.is_numeric_dtype(df[c])]
cat_cols_all = [c for c in common_cols if (df[c].dtype == "object" or str(df[c].dtype).startswith("category"))]

# =========================
# Tabs (7 secciones)
# =========================
tabs = st.tabs([
    "1) 🧰 Extracción",
    "2) 🔠 Categóricas",
    "3) 📈 Regresión Lineal Simple",
    "4) 🧮 Regresión Lineal Múltiple",
    "5) 🌀 Regresión No Lineal",
    "6) 🧪 Regresión Logística",
    "7) 💡 Insights & Propuestas",
])

# ============================================================
# TAB 1 - Extracción (comparativos de variables de servicio)
# ============================================================
with tabs[0]:
    st.subheader("🧰 Extracción: comparativo de variables clave por ciudad")
    st.caption("Variables típicas: price, number_of_reviews, availability_365, calculated_host_listings_count, accommodates, reviews_per_month, minimum_nights, estimated_occupancy_l365d, review_scores_rating.")

    candidates = [
        "price", "number_of_reviews", "availability_365",
        "calculated_host_listings_count", "host_total_listings_count",
        "accommodates", "reviews_per_month", "minimum_nights",
        "estimated_occupancy_l365d", "review_scores_rating"
    ]
    present = [c for c in candidates if c in common_cols]
    sel_vars = st.multiselect("Variables a comparar", options=present, default=present)

    if not sel_vars:
        st.info("Selecciona al menos una variable.")
    else:
        # Conversión segura a numérico
        df_num = coerce_numeric_cols(df, sel_vars)

        # KPIs por ciudad (mediana y promedio)
        try:
            kpi = (df_num.groupby("city")[sel_vars]
                   .agg(['median', 'mean'])
                   .round(3))
            st.dataframe(kpi, use_container_width=True)
        except Exception as e:
            st.warning(f"No fue posible calcular median/mean: {e}")

        # Gráficas por variable (box + barras) con columnas numéricas
        for v in sel_vars:
            st.markdown(f"**Variable:** `{v}`")
            c1, c2 = st.columns(2)

            with c1:
                try:
                    fig = px.box(df_num, x="city", y=v, points="outliers", color="city",
                                 title=f"Distribución de {v} por ciudad")
                    st.plotly_chart(fig, use_container_width=True)
                except Exception:
                    st.info(f"No se pudo graficar boxplot para {v}.")

            with c2:
                try:
                    # SeriesGroupBy.median sin numeric_only (ya es numérico)
                    agg = df_num.groupby("city")[v].median().reset_index()
                    fig2 = px.bar(agg, x="city", y=v, title=f"Mediana de {v} por ciudad")
                    st.plotly_chart(fig2, use_container_width=True)
                except Exception:
                    st.info(f"No se pudo graficar barras para {v}.")

# ============================================================
# TAB 2 - Categóricas (10 comunes)
# ============================================================
with tabs[1]:
    st.subheader("🔠 Análisis de variables categóricas (reglas específicas)")
    st.caption("room_type y host_is_superhost en todas; neighbourhood y host_response_time solo donde existan (no en México).")

    # --- Variables canónicas a usar ---
    canonical = ["room_type", "host_is_superhost", "neighbourhood", "host_response_time"]

    # Aliases para ubicar columnas equivalentes por nombre
    alias_map = {
        "room_type": ["room_type", "room", "tipo_habitacion"],
        "host_is_superhost": ["host_is_superhost", "is_superhost", "superhost"],
        "neighbourhood": ["neighbourhood", "neighborhood", "neighbourhood_cleansed", "barrio", "district", "zona"],
        "host_response_time": ["host_response_time", "response_time", "tiempo_respuesta"],
    }

    def find_col(df_local, aliases):
        cols = list(df_local.columns)
        low_cols = [c.lower() for c in cols]
        # Exacta (case-insensitive)
        for a in aliases:
            if a.lower() in low_cols:
                return cols[low_cols.index(a.lower())]
        # Inclusión parcial
        for a in aliases:
            for i, c in enumerate(low_cols):
                if a.lower() in c:
                    return cols[i]
        return None

    def to_yes_no(series):
        s = series.astype(str).str.lower().str.strip()
        yes = {"1","t","true","yes","si","sí","y","s"}
        no  = {"0","f","false","no","n"}
        return np.where(s.isin(yes), "Sí",
                        np.where(s.isin(no), "No", series.astype(str)))

    # --- Construcción del DataFrame categórico unificado ---
    cat_frames = []
    mapping_rows = []

    for d in dfs_raw:
        if d is None or "city" not in d.columns or d.empty:
            continue

        tmp = pd.DataFrame()
        tmp["city"] = d["city"].astype(str)

        # Detecta si el DF es de México (para excluir neighbourhood/response_time)
        df_has_mexico = tmp["city"].astype(str).str.lower().str.contains("mexico").any()

        # 1. room_type → siempre
        src_room = find_col(d, alias_map["room_type"])
        tmp["room_type"] = d[src_room] if src_room else np.nan

        # 2. host_is_superhost → siempre (convertido a Sí/No)
        src_super = find_col(d, alias_map["host_is_superhost"])
        if src_super:
            tmp["host_is_superhost"] = pd.Series(to_yes_no(d[src_super])).astype("category")
        else:
            tmp["host_is_superhost"] = np.nan

        # 3. neighbourhood → solo donde exista y no sea México
        src_neigh = find_col(d, alias_map["neighbourhood"])
        if (not df_has_mexico) and src_neigh:
            tmp["neighbourhood"] = d[src_neigh]
        else:
            tmp["neighbourhood"] = np.nan

        # 4. host_response_time → solo donde exista y no sea México
        src_resp = find_col(d, alias_map["host_response_time"])
        if (not df_has_mexico) and src_resp:
            tmp["host_response_time"] = d[src_resp]
        else:
            tmp["host_response_time"] = np.nan

        cat_frames.append(tmp)

        # Para mostrar en la tabla qué columnas se usaron
        cities_str = ", ".join(sorted(tmp["city"].astype(str).unique().tolist()))
        mapping_rows.append({
            "Ciudad(es) en DF": cities_str,
            "room_type <-": src_room if src_room else "❌",
            "host_is_superhost <-": src_super if src_super else "❌",
            "neighbourhood <-": (src_neigh if (src_neigh and not df_has_mexico) else "❌ (descartado o no existe)"),
            "host_response_time <-": (src_resp if (src_resp and not df_has_mexico) else "❌ (descartado o no existe)"),
        })

    if not cat_frames:
        st.warning("No se pudieron construir variables categóricas desde los datasets originales.")
        st.stop()

    df_cat_all = pd.concat(cat_frames, ignore_index=True)
    df_cat = df_cat_all[df_cat_all["city"].isin(sel_cities)].copy()

    if df_cat.empty:
        st.warning("No hay registros categóricos en las ciudades seleccionadas.")
        st.stop()

    # Mostrar mapeo de columnas detectadas por dataset/ciudad
    with st.expander("Ver mapeo de columnas utilizadas por dataset/ciudad"):
        st.dataframe(pd.DataFrame(mapping_rows), use_container_width=True)

    # Filtra solo las variables realmente disponibles con datos
    present_cats = [c for c in canonical if c in df_cat.columns and df_cat[c].notna().any()]

    # Selector para activar/desactivar variables
    sel_cats = st.multiselect(
        "Selecciona variables a visualizar",
        options=present_cats,
        default=present_cats,
        max_selections=len(present_cats)
    )

    if not sel_cats:
        st.info("Selecciona al menos una variable categórica.")
    else:
        for cat in sel_cats:
            st.markdown(f"### 📊 Variable: `{cat}`")

            # --- Distribución global ---
            try:
                dist = df_cat[cat].astype("category").value_counts(dropna=False).reset_index()
                dist.columns = [cat, "Frecuencia"]

                col1, col2 = st.columns(2)
                with col1:
                    fig_bar = px.bar(
                        dist, x=cat, y="Frecuencia", color=cat,
                        title=f"Frecuencia global de {cat}"
                    )
                    st.plotly_chart(fig_bar, use_container_width=True)

                with col2:
                    fig_pie = px.pie(
                        dist, names=cat, values="Frecuencia",
                        title=f"Participación global de {cat}"
                    )
                    st.plotly_chart(fig_pie, use_container_width=True)

            except Exception as e:
                st.warning(f"No se pudo graficar {cat}: {e}")

            # --- Comparativo por ciudad ---
            try:
                cross = (
                    df_cat.groupby(["city", cat])
                          .size()
                          .reset_index(name="Conteo")
                )
                fig_cross = px.bar(
                    cross, x="city", y="Conteo", color=cat,
                    barmode="group", title=f"{cat} por ciudad"
                )
                st.plotly_chart(fig_cross, use_container_width=True)
            except Exception as e:
                st.warning(f"No se pudo generar el comparativo de {cat} por ciudad: {e}")

        st.success("✅ Tab 2 configurado según reglas: room_type y superhost en todas; neighbourhood y response_time solo donde existan (no México).")

# ============================================================
# TAB 3 - Regresión Lineal Simple (facet por ciudad, Plotly ampliado)
# ============================================================
with tabs[2]:
    st.subheader("📐 Regresión Lineal Simple (comparativo multi-ciudad)")
    st.caption("Selecciona X e Y; se muestran todas las ciudades en facetas, con línea de ajuste y métricas R² y R.")

    # Ciudades
    try:
        cities_sel = sel_cities
    except NameError:
        cities_sel = sorted(df_all["city"].dropna().unique().tolist())

    # Definición segura de X e Y
    if "x_var" not in locals() or "y_var" not in locals():
        num_cols_all = df_all.select_dtypes(include=["int", "float", "Int64", "Float64"]).columns.tolist()
        fallback_numeric = ["price", "number_of_reviews", "availability_365", "reviews_per_month",
                            "calculated_host_listings_count", "host_total_listings_count"]
        for col in fallback_numeric:
            if col in df_all.columns and col not in num_cols_all:
                df_all[col] = pd.to_numeric(df_all[col], errors="coerce")
        num_cols_all = df_all.select_dtypes(include=["int", "float", "Int64", "Float64"]).columns.tolist()
        x_default = "availability_365" if "availability_365" in num_cols_all else (num_cols_all[0] if num_cols_all else None)
        y_default = "price" if "price" in num_cols_all else (num_cols_all[1] if len(num_cols_all) > 1 else None)
        x_var, y_var = x_default, y_default

    num_cols_ui = df_all.select_dtypes(include=["int", "float", "Int64", "Float64"]).columns.tolist()
    if not num_cols_ui:
        st.warning("No se detectaron columnas numéricas para realizar la regresión.")
        st.stop()

    x_var = st.selectbox("Variable X (independiente)", options=num_cols_ui,
                         index=max(0, num_cols_ui.index(x_var) if x_var in num_cols_ui else 0),
                         key="tab3_x_var")
    y_var = st.selectbox("Variable Y (dependiente)", options=num_cols_ui,
                         index=max(0, num_cols_ui.index(y_var) if y_var in num_cols_ui else (1 if len(num_cols_ui)>1 else 0)),
                         key="tab3_y_var")

    if x_var == y_var:
        st.info("Selecciona variables distintas para X e Y.")
        st.stop()

    # Datos
    data_lin = df_all[df_all["city"].isin(cities_sel)][["city", x_var, y_var]].copy()
    data_lin[x_var] = pd.to_numeric(data_lin[x_var], errors="coerce")
    data_lin[y_var] = pd.to_numeric(data_lin[y_var], errors="coerce")
    data_lin = data_lin.dropna(subset=[x_var, y_var])
    if data_lin.empty:
        st.warning("No hay datos válidos para las variables seleccionadas en las ciudades elegidas.")
        st.stop()

    # Ajuste por ciudad
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score
    metrics_rows, preds_rows = [], []
    for c in sorted(data_lin["city"].dropna().unique()):
        d = data_lin[data_lin["city"] == c]
        if d.shape[0] < 2: continue
        X, y = d[[x_var]].values, d[y_var].values
        lr = LinearRegression().fit(X, y)
        yhat = lr.predict(X)
        r2 = r2_score(y, yhat)
        try:
            corr = np.corrcoef(d[x_var], d[y_var])[0, 1]
            sign = np.sign(corr) if np.isfinite(corr) else 1
        except Exception:
            sign = 1
        R = sign * np.sqrt(max(r2, 0))
        metrics_rows.append({"city": c, "R2": round(r2, 4), "R": round(R, 4)})
        xx = np.linspace(d[x_var].min(), d[x_var].max(), 120).reshape(-1, 1)
        yy = lr.predict(xx)
        preds_rows.append(pd.DataFrame({"city": c, x_var: xx.ravel(), "y_pred": yy}))

    metrics_df = pd.DataFrame(metrics_rows)
    preds_df = pd.concat(preds_rows, ignore_index=True) if preds_rows else pd.DataFrame()
    if metrics_df.empty:
        st.warning("No fue posible ajustar modelos por falta de datos suficientes.")
        st.stop()

    # Visualización facetada grande
    fig_sc = px.scatter(data_lin, x=x_var, y=y_var, facet_col="city", facet_col_wrap=3,
                        title=f"Regresión Lineal Simple: {y_var} ~ {x_var}",
                        opacity=0.75, height=900, width=1500)
    if not preds_df.empty:
        fig_line = px.line(preds_df, x=x_var, y="y_pred", facet_col="city", facet_col_wrap=3)
        for tr in fig_line.data: fig_sc.add_trace(tr)
    fig_sc.update_traces(marker=dict(size=5))
    fig_sc.update_layout(margin=dict(l=40, r=40, t=60, b=40))
    st.plotly_chart(fig_sc, use_container_width=False)

    st.markdown("#### 📊 Métricas por ciudad")
    st.dataframe(metrics_df.sort_values("R2", ascending=False).reset_index(drop=True), use_container_width=True)
    
# ============================================================
# TAB 4 - Regresión Múltiple (facet por ciudad, Plotly ampliado)
# ============================================================
with tabs[3]:
    st.subheader("🧮 Regresión Múltiple (comparativo multi-ciudad)")
    st.caption("Selecciona Y y las X; facetas por ciudad (y_real vs y_pred) con métricas R² y R.")

    try:
        cities_sel = sel_cities
    except NameError:
        cities_sel = sorted(df_all["city"].dropna().unique().tolist())

    num_cols_all = df_all.select_dtypes(include=["int", "float", "Int64", "Float64"]).columns.tolist()
    if not num_cols_all:
        st.warning("No hay columnas numéricas para ajustar una regresión múltiple.")
        st.stop()

    try:
        y_var
    except NameError:
        y_var = "price" if "price" in num_cols_all else num_cols_all[0]
    try:
        x_vars
    except NameError:
        x_vars = [c for c in num_cols_all if c != y_var][:3] or num_cols_all[:3]

    y_var = st.selectbox("Variable Y (dependiente)", options=num_cols_all,
                         index=max(0, num_cols_all.index(y_var) if y_var in num_cols_all else 0),
                         key="tab4_y_var")
    x_vars = st.multiselect("Variables X (explicativas)", options=[c for c in num_cols_all if c != y_var],
                            default=[c for c in x_vars if c in num_cols_all and c != y_var],
                            key="tab4_x_vars")
    if not x_vars:
        st.info("Selecciona al menos una variable explicativa (X).")
        st.stop()

    cols_needed = ["city", y_var] + x_vars
    data_mlr = df_all[df_all["city"].isin(cities_sel)][cols_needed].copy()
    for c in [y_var] + x_vars:
        data_mlr[c] = pd.to_numeric(data_mlr[c], errors="coerce")
    data_mlr = data_mlr.dropna(subset=[y_var] + x_vars)
    if data_mlr.empty or data_mlr.shape[0] < len(x_vars) + 2:
        st.warning("No hay suficientes datos válidos para ajustar la regresión múltiple.")
        st.stop()

    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score
    metrics_rows, avp_rows = [], []
    for c in sorted(data_mlr["city"].dropna().unique()):
        d = data_mlr[data_mlr["city"] == c].copy()
        if d.shape[0] < len(x_vars) + 2: continue
        X, y = d[x_vars].values, d[y_var].values
        lr = LinearRegression().fit(X, y)
        yhat = lr.predict(X)
        r2 = r2_score(y, yhat)
        corr = np.corrcoef(y, yhat)[0,1] if len(y) > 1 else np.nan
        sign = np.sign(corr) if np.isfinite(corr) else 1
        R = sign * np.sqrt(max(r2, 0))
        metrics_rows.append({"city": c, "R2": round(r2, 4), "R": round(R, 4)})
        avp_rows.append(pd.DataFrame({"city": c, "y_real": y, "y_pred": yhat}))

    metrics_df = pd.DataFrame(metrics_rows)
    avp_df = pd.concat(avp_rows, ignore_index=True) if avp_rows else pd.DataFrame()
    if avp_df.empty:
        st.info("Sin suficientes datos por ciudad para graficar.")
        st.stop()

    fig_mlr = px.scatter(avp_df, x="y_real", y="y_pred",
                         facet_col="city", facet_col_wrap=3,
                         title=f"Regresión Múltiple: {y_var} vs Predicción (por ciudad)",
                         opacity=0.75, height=900, width=1500)
    fig_mlr.update_traces(marker=dict(size=5))
    fig_mlr.update_layout(margin=dict(l=40, r=40, t=60, b=40))
    st.plotly_chart(fig_mlr, use_container_width=False)

    st.markdown("#### 📊 Métricas por ciudad")
    st.dataframe(metrics_df.sort_values("R2", ascending=False).reset_index(drop=True), use_container_width=True)
    
# ============================================================
# TAB 5 - Regresión No Lineal (facet por ciudad, Plotly ampliado)
# ============================================================
with tabs[4]:
    st.subheader("🧭 Regresión No Lineal (comparativo multi-ciudad)")
    st.caption("Selecciona X, Y y el modelo no lineal; facetas por ciudad, línea de ajuste y métricas R² y R.")

    try:
        cities_sel = sel_cities
    except NameError:
        cities_sel = sorted(df_all["city"].dropna().unique().tolist())

    num_cols_all = df_all.select_dtypes(include=["int", "float", "Int64", "Float64"]).columns.tolist()
    if not num_cols_all:
        st.warning("No hay columnas numéricas para ajustar una regresión no lineal.")
        st.stop()

    try:
        x_var
    except NameError:
        x_var = "availability_365" if "availability_365" in num_cols_all else num_cols_all[0]
    try:
        y_var
    except NameError:
        y_var = "price" if "price" in num_cols_all else (num_cols_all[1] if len(num_cols_all)>1 else num_cols_all[0])
    modelos_disp = ["cuadrática", "logarítmica", "exponencial", "lineal"]
    try:
        modelo
    except NameError:
        modelo = "cuadrática"

    x_var = st.selectbox("Variable X (independiente)", options=num_cols_all,
                         index=max(0, num_cols_all.index(x_var) if x_var in num_cols_all else 0),
                         key="tab5_x_var")
    y_var = st.selectbox("Variable Y (dependiente)", options=num_cols_all,
                         index=max(0, num_cols_all.index(y_var) if y_var in num_cols_all else (1 if len(num_cols_all)>1 else 0)),
                         key="tab5_y_var")
    modelo = st.selectbox("Modelo", options=modelos_disp,
                          index=modelos_disp.index(modelo) if modelo in modelos_disp else 0,
                          key="tab5_modelo")

    if x_var == y_var:
        st.info("Selecciona variables distintas para X e Y.")
        st.stop()

    data_nl = df_all[df_all["city"].isin(cities_sel)][["city", x_var, y_var]].copy()
    data_nl[x_var] = pd.to_numeric(data_nl[x_var], errors="coerce")
    data_nl[y_var] = pd.to_numeric(data_nl[y_var], errors="coerce")
    data_nl = data_nl.dropna(subset=[x_var, y_var])
    if data_nl.empty:
        st.warning("No hay datos válidos para las variables seleccionadas.")
        st.stop()

    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score
    metrics_rows, preds_rows = [], []

    def fit_predict_model(d, modelo_sel):
        X = d[[x_var]].values
        y = d[y_var].values
        if modelo_sel == "cuadrática":
            X_ = np.c_[X, X**2]; lr = LinearRegression().fit(X_, y); return lr, (lambda xx: lr.predict(np.c_[xx, xx**2]))
        elif modelo_sel == "logarítmica":
            if (d[x_var] <= 0).any(): return None, None
            X_ = np.log(X); lr = LinearRegression().fit(X_, y); return lr, (lambda xx: lr.predict(np.log(xx)))
        elif modelo_sel == "exponencial":
            if (d[y_var] <= 0).any(): return None, None
            lr = LinearRegression().fit(X, np.log(y)); return lr, (lambda xx: np.exp(lr.predict(xx)))
        else:
            lr = LinearRegression().fit(X, y); return lr, (lambda xx: lr.predict(xx))

    for c in sorted(data_nl["city"].dropna().unique()):
        d = data_nl[data_nl["city"] == c].copy()
        if d.shape[0] < 2: continue
        lr, f_pred = fit_predict_model(d, modelo)
        if lr is None: continue
        X = d[[x_var]].values; y = d[y_var].values; yhat = f_pred(X)
        r2 = r2_score(y, yhat)
        corr = np.corrcoef(d[x_var], y)[0,1] if len(d)>1 else np.nan
        sign = np.sign(corr) if np.isfinite(corr) else 1
        R = sign * np.sqrt(max(r2, 0))
        metrics_rows.append({"city": c, "R2": round(r2,4), "R": round(R,4)})
        xx = np.linspace(d[x_var].min(), d[x_var].max(), 120).reshape(-1,1)
        yy = f_pred(xx)
        preds_rows.append(pd.DataFrame({"city": c, x_var: xx.ravel(), "y_pred": yy}))

    metrics_df = pd.DataFrame(metrics_rows)
    preds_df = pd.concat(preds_rows, ignore_index=True) if preds_rows else pd.DataFrame()

    fig_sc = px.scatter(data_nl, x=x_var, y=y_var, facet_col="city", facet_col_wrap=3,
                        title=f"Regresión No Lineal ({modelo}): {y_var} ~ {x_var}",
                        opacity=0.75, height=900, width=1500)
    if not preds_df.empty:
        fig_line = px.line(preds_df, x=x_var, y="y_pred",
                           facet_col="city", facet_col_wrap=3)
        for tr in fig_line.data: fig_sc.add_trace(tr)
    fig_sc.update_traces(marker=dict(size=5))
    fig_sc.update_layout(margin=dict(l=40, r=40, t=60, b=40))
    st.plotly_chart(fig_sc, use_container_width=False)

    st.markdown("#### 📊 Métricas por ciudad")
    if metrics_df.empty:
        st.info("No se pudieron calcular métricas (revisa datos/modelo).")
    else:
        st.dataframe(metrics_df.sort_values("R2", ascending=False).reset_index(drop=True), use_container_width=True)
        
# ============================================================
# TAB 6 - Regresión Logística (facet por ciudad, Plotly ampliado)
# ============================================================
with tabs[5]:
    st.subheader("🧪 Regresión Logística (comparativo multi-ciudad)")
    st.caption("Selecciona la Y binaria y las X; curvas ROC por ciudad en facetas y métricas AUC/Precision/Recall/F1.")

    try:
        cities_sel = sel_cities
    except NameError:
        cities_sel = sorted(df_all["city"].dropna().unique().tolist())

    # Y binaria y X_vars seguros
    try:
        y_bin
    except NameError:
        y_bin = "host_is_superhost" if "host_is_superhost" in df_all.columns else None

    num_cols_all = df_all.select_dtypes(include=["int", "float", "Int64", "Float64"]).columns.tolist()
    try:
        X_vars
    except NameError:
        X_vars = [c for c in ["price","availability_365","number_of_reviews"] if c in num_cols_all] or num_cols_all[:2]

    # Selector de Y binaria
    if y_bin is None or y_bin not in df_all.columns:
        bin_candidates = [c for c in df_all.columns if df_all[c].dropna().astype(str).str.lower().isin(["0","1","t","f","true","false","yes","no","si","sí"]).any()]
        if not bin_candidates:
            st.warning("No se detectó una columna binaria para Y. Define y_bin o crea una binarización en tu Tab de Logística.")
            st.stop()
        y_bin = st.selectbox("Variable Y binaria", options=bin_candidates, index=0, key="tab6_y_bin")
    else:
        all_opts = [y_bin] + [c for c in df_all.columns if c != y_bin]
        y_bin = st.selectbox("Variable Y binaria", options=all_opts, index=0, key="tab6_y_bin_fixed")

    # Select Xs
    X_vars = st.multiselect("Variables X (explicativas)", options=[c for c in num_cols_all if c != y_bin],
                            default=[x for x in X_vars if x in num_cols_all and x != y_bin],
                            key="tab6_x_vars")
    if not X_vars:
        st.info("Selecciona al menos una variable explicativa (X).")
        st.stop()

    # Parámetros de entrenamiento
    try:
        class_weight_opt
    except NameError:
        class_weight_opt = "balanced"
    try:
        threshold_opt
    except NameError:
        threshold_opt = 0.5

    class_weight_opt = st.selectbox("class_weight", options=["none","balanced"],
                                    index=1 if class_weight_opt in ["balanced"] else 0,
                                    key="tab6_class_weight")
    threshold_opt = st.number_input("Umbral de clasificación (0–1)", min_value=0.0, max_value=1.0,
                                    value=float(threshold_opt), step=0.05, key="tab6_threshold")

    # Preparación de datos
    cols_needed = ["city", y_bin] + X_vars
    data_log = df_all[df_all["city"].isin(cities_sel)][cols_needed].copy()

    # Y binaria robusta
    y_raw = data_log[y_bin].astype(str).str.strip().str.lower()
    y_num = np.where(y_raw.isin(["1","t","true","yes","si","sí","y","s"]), 1,
                     np.where(y_raw.isin(["0","f","false","no","n"]), 0, np.nan))
    data_log[y_bin] = pd.to_numeric(y_num, errors="coerce")

    # X numérico
    for c in X_vars:
        data_log[c] = pd.to_numeric(data_log[c], errors="coerce")

    data_log = data_log.dropna(subset=[y_bin] + X_vars)
    if data_log.empty or data_log[y_bin].nunique() < 2:
        st.warning("No hay suficientes datos/variabilidad en la Y binaria para ajustar la logística.")
        st.stop()

    # Ajuste por ciudad + ROC facet
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_fscore_support

    metrics_rows, roc_rows = [], []
    for c in sorted(data_log["city"].dropna().unique()):
        d = data_log[data_log["city"] == c].copy()
        if d[y_bin].nunique() < 2 or d.shape[0] < len(X_vars) + 2: continue

        X, y = d[X_vars].values, d[y_bin].astype(int).values
        cw = None if class_weight_opt in ["none", None, ""] else class_weight_opt
        lr = LogisticRegression(max_iter=500, class_weight=cw)
        lr.fit(X, y)
        proba = lr.predict_proba(X)[:, 1]

        fpr, tpr, _ = roc_curve(y, proba)
        auc = roc_auc_score(y, proba)
        roc_rows.append(pd.DataFrame({"city": c, "fpr": fpr, "tpr": tpr, "auc": auc}))

        thr = float(threshold_opt)
        yhat = (proba >= thr).astype(int)
        prec, rec, f1, _ = precision_recall_fscore_support(y, yhat, average="binary", zero_division=0)
        metrics_rows.append({"city": c, "AUC": round(auc,4), "Precision": round(prec,4), "Recall": round(rec,4), "F1": round(f1,4)})

    roc_df = pd.concat(roc_rows, ignore_index=True) if roc_rows else pd.DataFrame()
    metrics_df = pd.DataFrame(metrics_rows) if metrics_rows else pd.DataFrame(columns=["city","AUC","Precision","Recall","F1"])

    if roc_df.empty:
        st.info("No hay suficientes datos para graficar ROC por ciudad.")
    else:
        fig_roc = px.area(roc_df, x="fpr", y="tpr",
                          facet_col="city", facet_col_wrap=3,
                          title="Regresión Logística: Curvas ROC por ciudad",
                          hover_data={"auc":":.3f"}, height=900, width=1500)
        fig_roc.update_layout(margin=dict(l=40, r=40, t=60, b=40))
        st.plotly_chart(fig_roc, use_container_width=False)

    st.markdown("#### 📊 Métricas por ciudad")
    st.dataframe(metrics_df.sort_values("AUC", ascending=False).reset_index(drop=True),
                 use_container_width=True)

# ============================================================
# TAB 7 - Insights & Propuestas
# ============================================================
with tabs[6]:
    st.subheader("📊 Insights y Recomendaciones Estratégicas")
    st.caption("Comparación de medias entre ciudades usando variables reconstruidas desde los datasets originales (sin intersección). Incluye % de Superhosts (T).")

    import re, unicodedata

    # ---- Utilidades robustas ----
    def _norm_txt(s: str) -> str:
        s = str(s)
        s = unicodedata.normalize("NFKD", s)
        s = "".join(ch for ch in s if not unicodedata.combining(ch))
        return s.strip().lower()

    def is_mex(name: str) -> bool:
        n = _norm_txt(name)
        return bool(
            re.search(r"^mex", n) or
            re.search(r"ciudad\s*de\s*mex", n) or
            re.search(r"\bcdmx\b", n) or
            re.fullmatch(r"mx", n)
        )

    def find_col(df_local, aliases):
        cols = list(df_local.columns)
        low = [c.strip().lower() for c in cols]
        # exacta
        for a in aliases:
            a2 = a.strip().lower()
            if a2 in low:
                return cols[low.index(a2)]
        # inclusión parcial
        for a in aliases:
            a2 = a.strip().lower()
            for i, c in enumerate(low):
                if a2 in c:
                    return cols[i]
        return None

    def clean_numeric_like(series):
        s = series.astype(str).str.replace(r"[^\d,.\-]", "", regex=True)
        mask_coma_decimal = ~s.str.contains(r"\.") & s.str.contains(",")
        s.loc[mask_coma_decimal] = s.loc[mask_coma_decimal].str.replace(",", ".", regex=False)
        s = s.str.replace(",", "", regex=False)
        return pd.to_numeric(s, errors="coerce")

    # ---- Aliases por variable ----
    alias = {
        "host_is_superhost": ["host_is_superhost", "is_superhost", "superhost"],
        "host_response_time": ["host_response_time", "response_time", "tiempo_respuesta"],
        "calculated_host_listings_count": ["calculated_host_listings_count", "host_total_listings_count"],
        "price": ["price", "precio"],
        "number_of_reviews": ["number_of_reviews", "reviews"],
        "availability_365": ["availability_365"],
    }

    # ---- Reconstrucción desde dfs_raw ----
    recs = []       # filas unificadas
    mappings = []   # mapeo por dataset

    for d in dfs_raw:
        if d is None or d.empty or "city" not in d.columns:
            continue
        df_local = d.copy()
        df_local.columns = [str(c).strip() for c in df_local.columns]

        tmp = pd.DataFrame()
        tmp["city"] = df_local["city"].astype(str)

        # host_is_superhost (con regla para México)
        col_hsh = find_col(df_local, alias["host_is_superhost"])
        if col_hsh:
            raw = df_local[col_hsh].astype(str).str.strip().str.lower()
            # máscara México
            mex_mask = tmp["city"].map(is_mex).fillna(False)

            # México: SOLO T/F/1/0 -> map estricta; resto NaN
            map_mex = {"t": 1, "f": 0, "1": 1, "0": 0}
            tmp.loc[mex_mask, "host_is_superhost_num"] = raw[mex_mask].map(map_mex)

            # Otras ciudades: mapeo flexible clásico
            oth = ~mex_mask
            tmp.loc[oth, "host_is_superhost_num"] = np.where(
                raw[oth].isin(["t","true","1","yes","si","sí","y","s"]), 1,
                np.where(raw[oth].isin(["f","false","0","no","n"]), 0, np.nan)
            )

            # También versión F/T para diagnóstico y visual
            tmp["host_is_superhost_str"] = tmp["host_is_superhost_num"].map({1.0: "T", 0.0: "F"})
        else:
            tmp["host_is_superhost_num"] = np.nan
            tmp["host_is_superhost_str"] = np.nan

        # host_response_time -> ordinal 4..1
        col_rt = find_col(df_local, alias["host_response_time"])
        if col_rt:
            m = {
                "within an hour": 4, "within a few hours": 3,
                "within a day": 2, "a few days or more": 1
            }
            tmp["host_response_time"] = (
                df_local[col_rt].astype(str).str.strip().str.lower().map(m)
            )
        else:
            tmp["host_response_time"] = np.nan

        # calculated_host_listings_count
        col_chl = find_col(df_local, alias["calculated_host_listings_count"])
        if col_chl:
            tmp["calculated_host_listings_count"] = pd.to_numeric(df_local[col_chl], errors="coerce")
        else:
            tmp["calculated_host_listings_count"] = np.nan

        # price (limpieza general; NaN se ignoran en medias; caso Valencia ya cubierto)
        col_price = find_col(df_local, alias["price"])
        if col_price:
            tmp["price"] = clean_numeric_like(df_local[col_price])
        else:
            tmp["price"] = np.nan

        # number_of_reviews
        col_nor = find_col(df_local, alias["number_of_reviews"])
        if col_nor:
            tmp["number_of_reviews"] = pd.to_numeric(df_local[col_nor], errors="coerce")
        else:
            tmp["number_of_reviews"] = np.nan

        # availability_365
        col_av = find_col(df_local, alias["availability_365"])
        if col_av:
            tmp["availability_365"] = pd.to_numeric(df_local[col_av], errors="coerce")
        else:
            tmp["availability_365"] = np.nan

        recs.append(tmp)

        # mapeo informativo
        maps = {
            "Ciudad(es) en DF": ", ".join(sorted(tmp["city"].astype(str).unique())),
            "host_is_superhost <-": col_hsh if col_hsh else "❌",
            "host_response_time <-": col_rt if col_rt else "❌",
            "calculated_host_listings_count <-": col_chl if col_chl else "❌",
            "price <-": col_price if col_price else "❌",
            "number_of_reviews <-": col_nor if col_nor else "❌",
            "availability_365 <-": col_av if col_av else "❌",
        }
        mappings.append(maps)

    if not recs:
        st.warning("No hay datos disponibles para generar insights.")
        st.stop()

    df_ins = pd.concat(recs, ignore_index=True)
    # Filtra por las ciudades seleccionadas
    df_ins = df_ins[df_ins["city"].isin(sel_cities)].copy()

    # ---- Diagnóstico de mapeos ----
    with st.expander("🔎 Mapeo de columnas utilizadas por dataset"):
        st.dataframe(pd.DataFrame(mappings), use_container_width=True)

    # ---- Tabla de diagnóstico Superhosts F/T por ciudad ----
    with st.expander("🔍 Distribución F/T por ciudad (host_is_superhost)"):
        if "host_is_superhost_str" in df_ins.columns:
            diag = (
                df_ins.groupby(["city", "host_is_superhost_str"])
                .size()
                .unstack(fill_value=0)
                .rename(columns={"F": "F (No)", "T": "T (Sí)"})
            )
            st.dataframe(diag, use_container_width=True)

    # ---- Cálculo de medias por ciudad ----
    metrics = ["host_is_superhost_num", "host_response_time",
               "calculated_host_listings_count", "price",
               "number_of_reviews", "availability_365"]

    summary = {}
    for v in metrics:
        if v in df_ins.columns:
            df_ins[v] = pd.to_numeric(df_ins[v], errors="coerce")
            mean_v = df_ins.groupby("city")[v].mean().round(2)
            if mean_v.notna().any():
                summary[v] = mean_v

    if not summary:
        st.warning("No fue posible calcular medias (todas las series vacías).")
        st.stop()

    df_summary = pd.DataFrame(summary).sort_index()

    # Porcentaje de Superhosts (T) por ciudad
    if "host_is_superhost_num" in df_summary.columns:
        df_summary["%_superhosts"] = (df_summary["host_is_superhost_num"] * 100).round(1)

    st.markdown("### 📈 Comparativo de medias entre ciudades")
    st.dataframe(df_summary, use_container_width=True)

    # ---- Gráficas ----
    # Hosts
    st.markdown("#### 🧑‍💼 Métricas de Hosts")
    for v, title in [
        ("host_is_superhost_num", "% Superhosts (promedio 0/1)"),
        ("host_response_time", "Rapidez de respuesta (4=mejor)"),
        ("calculated_host_listings_count", "Propiedades por host (promedio)"),
    ]:
        if v in df_summary.columns:
            fig = px.bar(df_summary.reset_index(), x="city", y=v, color="city",
                         title=f"{title} — promedio por ciudad")
            st.plotly_chart(fig, use_container_width=True)

    # Hospedaje
    st.markdown("#### 🏠 Métricas de Hospedaje")
    for v, title in [
        ("price", "Precio promedio"),
        ("number_of_reviews", "Reseñas promedio"),
        ("availability_365", "Disponibilidad promedio (días)"),
    ]:
        if v in df_summary.columns:
            fig = px.bar(df_summary.reset_index(), x="city", y=v, color="city",
                         title=f"{title} — promedio por ciudad")
            st.plotly_chart(fig, use_container_width=True)

    # ---- Recomendaciones ----
    st.markdown("### 🧭 Recomendaciones Estratégicas")
    recs = []
    if "%_superhosts" in df_summary.columns:
        minc = df_summary["%_superhosts"].idxmin()
        recs.append(f"- **{minc}**: impulsar la ruta a Superhost (mentoría + checklist de amenities) para elevar el % de superhosts.")
    if "host_response_time" in df_summary.columns:
        minr = df_summary["host_response_time"].idxmin()
        recs.append(f"- **{minr}**: activar respuestas automáticas y recordatorios para mejorar tiempos de respuesta de hosts.")
    if "availability_365" in df_summary.columns:
        maxa = df_summary["availability_365"].idxmax()
        recs.append(f"- **{maxa}**: aplicar pricing dinámico y promos de temporada baja para reducir vacancia anual.")
    if "price" in df_summary.columns:
        maxp = df_summary["price"].idxmax()
        recs.append(f"- **{maxp}**: consolidar oferta premium y low-budget (bundles: late checkout, limpieza) como estrategia comercial.")
    if "number_of_reviews" in df_summary.columns:
        minrv = df_summary["number_of_reviews"].idxmin()
        recs.append(f"- **{minrv}**: campañas de visibilidad + recordatorios post-estancia para aumentar reseñas.")
    if "calculated_host_listings_count" in df_summary.columns:
        maxh = df_summary["calculated_host_listings_count"].idxmax()
        recs.append(f"- **{maxh}**: estrategia de diversificación de alojamientos.")

    for r in recs:
        st.markdown(r)

    st.success("✅ host_is_superhost de MÉXICO tomado correctamente (solo T/F/1/0) y % de T por ciudad calculado.")

Overwriting EVIDENCIA_2.py
